#<font color=fbb03b>翌週のサザエさんのじゃんけんを予測（LightGBMモデルを使用）

##<font color=fbb03b>事前準備

Google Driveを使用の場合は以下のセルを実行してドライブをマウントし実行ディレクトリへ移動します.

In [ ]:
from google.colab import drive, output
output.clear()
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''データが置いてあるディレクトリのパスを入力して実行しましょう．'''
# 以下はサンプルです.ご自身の環境に合わせてフォルダへのパスは変更してください.
%cd /content/drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


###ライブラリのインポート

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from lightgbm import early_stopping

###データ準備

In [ ]:
# データ読み込み
df = pd.read_csv('traindata_サザエさんじゃんけん分析.xlsx')

In [ ]:
df.tail()

,放送回,放送日,出した手,備考
1647,1648,45515,グー,NaN
1648,1649,45522,チョキ,NaN
1649,1650,45529,グー,NaN
1650,1651,45536,チョキ,NaN
1651,1652,45543,NaN,NaN


##<font color=fbb03b>特徴量を準備

In [ ]:
# 日付情報の特徴量化
df['放送日'] = pd.to_datetime(df['放送日'])
df['年'] = df['放送日'].dt.year
df['月'] = df['放送日'].dt.month
df['日'] = df['放送日'].dt.day
df['曜日'] = df['放送日'].dt.weekday

In [ ]:
# ラベルエンコーディング before creating lagged features
le = LabelEncoder()
df['出した手_enc'] = le.fit_transform(df['出した手'])

In [ ]:
# 過去の手の特徴量化 (例: 過去1651回)
for i in range(1, 1651):
    df[f'{i}回前の手'] = df['出した手_enc'].shift(i).fillna(method='bfill').astype(int)  # Convert to integers after shifting

ストリーミング出力は最後の 5000 行に切り捨てられました。
<ipython-input-24-ad6e95b4d5ce>:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[f'{i}回前の手'] = df['出した手_enc'].shift(i).fillna(method='bfill').astype(int)  # Convert to integers after shifting
<ipython-input-24-ad6e95b4d5ce>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{i}回前の手'] = df['出した手_enc'].shift(i).fillna(method='bfill').astype(int)  # Convert to integers after shifting
<ipython-input-24-ad6e95b4d5ce>:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[f'{i}回前の手'] = df['出した手_enc'].shift(i).fillna(method='bfill').astype(int)  # Convert

##<font color=fbb03b>TrainDataとTestDataに分割

In [ ]:
# データ分割
X = df[['年', '月', '日', '曜日', '1回前の手', '2回前の手', '3回前の手']]
y = df['出した手_enc']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##<font color=fbb03b>LightGBMモデル構築

In [ ]:
# LightGBMモデル構築
model = LGBMClassifier(objective='multiclass', num_class=3, metric='multi_logloss', random_state=42)

###データの学習

In [ ]:
# 学習
model.fit(X_train, y_train,
          eval_set=[(X_test, y_test)], # Add validation set
          eval_metric='multi_logloss', # Specify evaluation metric
          callbacks=[early_stopping(10)])

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000205 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9
[LightGBM] [Info] Number of data points in the train set: 1321, number of used features: 3
[LightGBM] [Info] Start training from score -1.113100
[LightGBM] [Info] Start training from score -1.065847
[LightGBM] [Info] Start training from score -1.120036
[LightGBM] [Info] Start training from score -7.186144
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

LGBMClassifier(metric='multi_logloss', num_class=3, objective='multiclass',
               random_state=42)

###予測結果を格納

In [ ]:
# 予測
y_pred = model.predict_proba(X_test)

##<font color=fbb03b>予測結果の出力

In [ ]:
# 結果出力 (例: 2024年9月8日の予測)
pred_date = pd.to_datetime('2024-09-8')
pred_data = pd.DataFrame([[pred_date.year, pred_date.month, pred_date.day, pred_date.weekday(),
                        df['出した手_enc'].iloc[-1], df['出した手_enc'].iloc[-2], df['出した手_enc'].iloc[-3]]],
                        columns=['年', '月', '日', '曜日', '1回前の手', '2回前の手', '3回前の手'])
pred_proba = model.predict_proba(pred_data)

print(f"2024年9月8日 グー {pred_proba[0][0]*100:.1f}%　チョキ {pred_proba[0][1]*100:.1f}%　パー {pred_proba[0][2]*100:.1f}%")

2024年9月8日 グー 38.0%　チョキ 36.6%　パー 25.4%
